In [ ]:
# Before running this cell, replace <your_password> below with the
# actual password you used when deploying the Big Data Cluster.

newPeople = (spark
    .read
    .format("csv")
    .options(header="true", infer_schema="true")
    .load("/NewPeople"))

existingPeople = (spark
    .read
    .format("jdbc")
    .option("url", "jdbc:sqlserver://master-0.master-svc;databaseName=Staging")
    .option("dbtable", "dbo.ExistingPeople")
    .option("user", "admin")
    .option("password", "<your_password>")
    .load())

newPeople.show(10)
existingPeople.show(10)

unknownPeople = newPeople.join(existingPeople, newPeople.EmailAddress == existingPeople.EmailAddress, how="left_anti")

print(newPeople.count())
print(existingPeople.count())
print(unknownPeople.count())

(unknownPeople
    .select("FirstName", "LastName", "EmailAddress")
    .write
    .format("com.databricks.spark.csv")
    .options(header="true")
    .mode("overwrite")
    .save("/UnknownPeople"))
